# Replication, Auto failover, Reading and Writing to Replica Sets

## Connect to MongoDB 

In [ ]:
import pymongo
from pymongo import MongoClient 
from pprint import pprint

## Read/Write from a secondary

Let's connect to a secondary and try to read from it ... (please note you need to adpat 27018 to a port of a secondary in your replica set) 

In [ ]:
myclient = MongoClient('localhost', 27019)
replica_set_db = myclient['replica_set_db']
replica_set_col = replica_set_db["replica_set_col"]
docs = replica_set_col.find()
for doc in docs:
    pprint(doc)

Let's now try to write to a secondary 

In [ ]:
doc = { "university": "Horizon", "course": "Mongo DB" }
adoc = replica_set_col.insert_one(doc)

## Read/Write from the primary 

Let's try to connect to the primary and save a doc ...

In [ ]:
myclient = MongoClient('localhost', 27018)
replica_set_db = myclient['replica_set_db']
replica_set_col = replica_set_db["replica_set_col"]
doc = { "University": "Horizon", "course": "Mongo DB" }
adoc = replica_set_col.insert_one(doc)

Now let’s bring down that node and see what happens when we run our query again

In [ ]:
doc = { "University": "Horizon", "course": "Redis" }
adoc = replica_set_col.insert_one(doc)

## Read/Write from replica set 

Now let's restart the 27018 and connect to the replica set insead ... 

In [ ]:
myclient = MongoClient('localhost', 27019, replicaSet='rs0')
replica_set_db = myclient['replica_set_db']
replica_set_col = replica_set_db["replica_set_col"]
docs = replica_set_col.find()
for doc in docs: 
    pprint(doc)

Let's write to the replica set 

In [ ]:
doc = { "University": "Horizon", "course": "Redis" }
adoc = replica_set_col.insert_one(doc)
docs = replica_set_col.find()
for doc in docs: 
    pprint(doc)

let's bring the instance 27019 down and try to read/write 

In [ ]:
docs = replica_set_col.find()
for doc in docs: 
    pprint(doc)

## Read Preference 

Actually, when connected to a replica set, by default reading and writing are performed from the primary

In [ ]:
replica_set_col.read_preference

We can set read preference

In [ ]:
from pymongo import ReadPreference
myclient = MongoClient('localhost', 27019, replicaSet='rs0', read_preference=ReadPreference.SECONDARY)
replica_set_db = myclient['replica_set_db']
replica_set_col = replica_set_db["replica_set_col"]
replica_set_col.read_preference

- Replica-set members can be tagged according to any criteria you choose.
- When selecting a server for a read operation with maxStalenessSeconds, clients estimate how stale each secondary is by comparing the secondary's last write to that of the primary. The client will then direct the read operation to a secondary whose estimated lag is less than or equal to maxStalenessSeconds.
- With hedged reads, the mongos instances can route read operations to two replica set members per each queried shard and return results from the first respondent per shard.

## Enabling hedged reads

In [ ]:
from pymongo.read_preferences import Secondary
read_preference = Secondary(tag_sets=None, max_staleness=-1, hedge={'enabled': True})
myclient = MongoClient('localhost', 27019, replicaSet='rs0', read_preference=read_preference)
replica_set_db = myclient['replica_set_db']
replica_set_col = replica_set_db["replica_set_col"]
replica_set_col.read_preference 